# Adaptive Forgetting test
Date: 01-May-2021

Description:
Directional Adaptive Rate Forgettingのテスト。ターゲットが動いている場合、Bayesian Linear Regressionは、そのSufficient Statistics
を計算する際に、古いデータは忘れる必要がある。このプロセスを自動的に行う方法を考えてみる。

通常、Precision Matrixは、以下の式で更新される。

$\Lambda_{t|t} = \Lambda_{t|t-1} + \beta^{-1} x_t x_t^T$

あるいは、Woodbury matrix identityを使って、Covariance Matrixを直接以下の様に更新する。

$\Sigma_{t|t} = \Sigma_{t|t-1} - \frac{\Sigma_{t|t-1} x_t x_t^T \Sigma_{t|t-1}}{\beta + x_t^T \Sigma_{t|t-1} x_t} $

通常、ターゲットが不変の場合は、これらがそのまま次のサンプルの予測に用いられる。

$\Lambda_{t+1|t} = \Lambda_{t|t}$

$\Sigma_{t+1|t} = \Sigma_{t|t}$

ターゲットが変動する場合は、何らかの方法でこれらを更新する必要がある。 ここでは、以下の様な更新方法を試す。



In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.style.use('seaborn-darkgrid')
mpl.rcParams['figure.dpi']= 300

In [1]:
def gen_data(len, m=[-1.0, 2.0], r=[0.5, 0.5], var=0.01, type='sin'):
    """
    generate data (simulate target model)
    :param len:     length of data
    :param m:       mean value
    :param r:       range (only used in type=='sin')
    :param var:     variance of likelihood noise
    :param type:    model type ('sin', 'linear', 'const')
    :return:        x, y (input/output)
    """
    t = np.arange(len)
    if type is 'sin':
        a = m[0] + r[0] * np.sin(t * 2*np.pi / 100)
        b = m[1] + r[1] * np.cos(t * 2*np.pi / 100)
    elif type is 'linear':
        a =        t / 2000
        b = m[1] + t / 2000
    elif type is 'sin_linear':
        a = m[0] + r[0] * np.sin(t * 2 * np.pi / 200) + t / 2000
        b = m[1] + r[1] * np.cos(t * 2 * np.pi / 200) + t / 2000
    elif type is 'const':
        a = m[0] + t * 0.0
        b = m[1] + t * 0.0
    x = np.random.randn(len) * 10.0
    y_no_noise = a * x + b
    y = y_no_noise + np.random.randn(len) * np.sqrt(var)
    info = {'a': a, 'b': b, 'y_no_noise': y_no_noise}
    return x, y, info

class DARF:
    def __init__(self, x_dim=2, beta=0.1):
        self.beta = beta
        self.Sigma = np.eye(x_dim) * 10.
        self.mu = np.zeros(x_dim)

    def fit(self, x, y):
        gamma = 1/(self.beta + x[None,:] @ self.Sigma @ x[:,None])
        gain = self.Sigma @ x[:,None] * gamma
        err = y - np.dot(x, self.mu)
        self.mu += gain[:,0] * err
        self.Sigma -= gain @ gain.T / gamma
        return err

    def predict(self, x):
        y = np.dot(x, self.mu)
        return y

type =  'const' #'linear' #'const' #'sin' # 'sin_linear'
sim_len = 5000#
beta = 0.1
X,Y, info = gen_data(sim_len, var=beta, type=type)

darf = DARF(2, beta)

for n in range(sim_len):
    x = X[n]
    y = Y[n]
    inp = np.ones(2)
    inp[0] = x
    err = darf.fit(inp,y)
    print(err)


NameError: name 'np' is not defined

In [ ]:
fig, ax = plt.subplots(1, sharex=True, figsize=(6.4, 4.5))
ax.plot([1,2,3],[1,4,9], marker='o', linestyle='-', color='C2', label='data 1')
ax.set_xlabel('x axis label')
ax.set_ylabel('y axis label')
ax.set_xticks([1,2,3])
ax.set_xticklabels(['one', 'two', 'three'])
ax.legend()

plt.show()
fig.savefig('example_plot.eps')

In [ ]:
x = np.array([1,2,3])
x[:,None]
